# Library

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression

In [ ]:
# mynewlist contains the clinical features to be analyzed
# df is the data frame containing clinical features and birth labels

mean_aucs = []

for feature_name in list_of_clinical_features:

    X = df[ feature_name ].to_numpy().reshape(-1, 1)
    y = df[ 'birth_label' ].to_numpy().reshape(-1,1)

    # Run classifier with cross-validation and plot ROC curves
    cv = StratifiedKFold(n_splits= 10)
    #classifier = svm.SVC(kernel="linear", probability=True, random_state= 42)
    classifier = LogisticRegression(class_weight= 'balanced', max_iter=1000)

    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    
    fig, ax = plt.subplots()
    for i, (train, test) in enumerate(cv.split(X, y)):
        # fitting on training dataset
        classifier.fit(X[train], y[train])
        
        # testing on test dataset
        viz = RocCurveDisplay.from_estimator(
            classifier,
            X[test],
            y[test],
            name="ROC fold {}".format(i),
            alpha=0.3,
            lw=1,
            ax=ax,
        )
        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(viz.roc_auc)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    
    print( '{}: {}'.format(feature_name, mean_auc) )
    
    mean_aucs.append( mean_auc )